In [ ]:
# fetech actural image from url

df = spark.sql(f"SELECT * FROM InFreGen.pic WHERE state = '0' and taskID = {taskID} ")
display(df)

StatementMeta(, 9e244fcf-6464-4825-82b5-4250482275f0, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 366133c1-48ca-4aba-b45d-26a6adf9ab13)

StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d45d733c-75e5-46ac-91a1-6574a8525832)

StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d868fbd6-fc20-41bd-a01e-ede073fb7edc)

StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f56f1243-76fb-425c-a765-f1a0111b6c0a)

In [ ]:
data = df.select("taskID", "picID", "url", "sizeChoice").toPandas()
data

StatementMeta(, 9e244fcf-6464-4825-82b5-4250482275f0, 16, Finished, Available, Finished)

,taskID,picID,url,sizeChoice
0,14,3,"{""raw"": ""https://images.unsplash.com/photo-170...",small
1,14,4,"{""raw"": ""https://images.unsplash.com/photo-164...",small
2,14,2,"{""raw"": ""https://images.unsplash.com/photo-160...",small
3,14,1,"{""raw"": ""https://images.unsplash.com/photo-155...",small
4,14,0,"{""raw"": ""https://images.unsplash.com/photo-155...",small


StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 4, Finished, Available, Finished)

,taskID,picID,url,sizeChoice


StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 11, Finished, Available, Finished)

,taskID,picID,url,sizeChoice


StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 18, Finished, Available, Finished)

,taskID,picID,url,sizeChoice
0,14,0,"{""raw"": ""https://images.unsplash.com/photo-155...",small
1,14,1,"{""raw"": ""https://images.unsplash.com/photo-155...",small
2,14,2,"{""raw"": ""https://images.unsplash.com/photo-160...",small
3,14,3,"{""raw"": ""https://images.unsplash.com/photo-170...",small
4,14,4,"{""raw"": ""https://images.unsplash.com/photo-164...",small


In [ ]:
import os
import requests

def download_image(url, save_path):
    # Get the image content
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Ensure the directory for the save path exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        # Write the image content to the file
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Image successfully downloaded and saved to {save_path}")
        return True
    else:
        print(f"Failed to retrieve image from {url}. Status code: {response.status_code}")
        return False


StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 19, Finished, Available, Finished)

In [ ]:
from delta.tables import *
from pyspark.sql import SparkSession

def write_originalPicPath(taskID, picID, originalPicPath):
    delta_table = DeltaTable.forName(spark, "InFreGen.pic")
    # update the originalPicPath and curPicPath
    update_data = [(taskID, picID, originalPicPath, originalPicPath)]
    update_columns = ["taskID", "picID", "originalPicPath", "curPicPath"]
    # create a dataframe to update the originalPicPath and curPicPath
    update_df = spark.createDataFrame(update_data, update_columns)

    # use delta lake to update the originalPicPath and curPicPath
    delta_table.alias("t") \
        .merge(
            update_df.alias("u"),
            "t.taskID = u.taskID AND t.picID = u.picID"  # condition is taskID and picID match
        ) \
        .whenMatchedUpdate(
            set={"t.originalPicPath": "u.originalPicPath",
                "t.curPicPath": "u.curPicPath"
            }  # update the originalPicPath and curPicPath
            
        ) \
        .execute()


StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 20, Finished, Available, Finished)

In [ ]:
import json

# download image
for index, row in data.iterrows():
    if(row["url"] == None or row["url"] == ""):
        continue
    # get the url choice
    url_choice = row["sizeChoice"]
    url = json.loads(row["url"])[url_choice]
    taskID = row["taskID"]
    picID = row["picID"]
    
    print("taskID" + str(taskID) + "url: " + str(url))

    # save the image to the lakehouse
    save_path = "/lakehouse/default/Files/images/" + str(taskID) + "/" + str(picID) + "_" + url_choice + ".jpg"


    # if download successful, write the url back to task table
    if(download_image(url, save_path)):
        write_originalPicPath(taskID, picID, save_path)



StatementMeta(, 99e8f7db-5e70-4b09-a5a8-ccc1029abd3b, 21, Finished, Available, Finished)

taskID14url: https://images.unsplash.com/photo-1557735065-ebd5a235f490?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2NjU3MTR8MHwxfHNlYXJjaHwxfHxzaGVlcHxlbnwwfDJ8fHwxNzMwNjcwMDY4fDA&ixlib=rb-4.0.3&q=80&w=400
Image successfully downloaded and saved to /lakehouse/default/Files/images/14/0_small.jpg
taskID14url: https://images.unsplash.com/photo-1551571303-d608e971994b?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2NjU3MTR8MHwxfHNlYXJjaHwyfHxzaGVlcHxlbnwwfDJ8fHwxNzMwNjcwMDY4fDA&ixlib=rb-4.0.3&q=80&w=400
Image successfully downloaded and saved to /lakehouse/default/Files/images/14/1_small.jpg
taskID14url: https://images.unsplash.com/photo-1601106099864-e24f574d3462?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w2NjU3MTR8MHwxfHNlYXJjaHwzfHxzaGVlcHxlbnwwfDJ8fHwxNzMwNjcwMDY4fDA&ixlib=rb-4.0.3&q=80&w=400
Image successfully downloaded and saved to /lakehouse/default/Files/images/14/2_small.jpg
taskID14url: https://images.unsplash.com/photo-1703473119372-d70bf30993e3?crop=entropy&cs=tinysrg

In [ ]:
# update task table
spark.sql("UPDATE InFreGen.task SET state = '1' WHERE taskID = {}".format(taskID))

In [ ]:
# update the pic table
spark.sql("UPDATE InFreGen.pic SET state = '1' WHERE taskID = {}".format(taskID))